In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
!git clone https://github.com/rafaseto/PIBIC-AnaliseDados.git

fatal: destination path 'PIBIC-AnaliseDados' already exists and is not an empty directory.


In [3]:
%cd PIBIC-AnaliseDados

[WinError 2] O sistema não pode encontrar o arquivo especificado: 'PIBIC-AnaliseDados'
c:\Users\rafae\Documents\somebooks\College\PIBIC\Rafael-AnaliseDados\PIBIC-AnaliseDados


In [1]:
import pip

In [2]:
!pip install dlisio


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from dlisio import dlis
import pandas as pd

In [4]:
# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

In [5]:
dli = []        # Conterá os arquivos lógicos
filenames = []  # Conterá os caminhos dos arquivos dlis
pocos = []      # Conterá strings identificadoras de 9 caracteres, tipo '3-CP-1847'
charId_inicio, charId_fim = 5, 14   # Índices de início e fim da substring identificadora do poço

# Utilizando a função 'glob' do módulo 'glob' para procurar todos os arquivos com extensão DLIS em 'Data'
for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    # Carregando os arquivos dlis com 'load' e armazenando-os em 'leitura'
    # 'tail' recebe valores restantes, caso a função retorne mais de uma peça de informação
    leitura, *tail = dlis.load(f'{file}')
    pocos.append(file[charId_inicio : charId_fim])    # armazenando identificadores do tipo '3-CP-XXXX'
    filenames.append(file)      # armazenando o caminho dos arquivos dlis
    dli.append(leitura)         # armazenando os arquivos lógicos carregados com 'load'

In [6]:
# Criando um dicionário para associar nomes aos elementos de dli
dli_dict = {
    'p-47': dli[0],
    'p-48': dli[1],
    'p-51': dli[2],
    'p-53': dli[3],
    'p-55': dli[4],
    'p-57': dli[5]
}

In [7]:
dli_dict['p-47'].describe()

------------
Logical File
------------
Description : LogicalFile(GEOLOAD.1)
Frames      : 1
Channels    : 21

Known objects
--
FRAME       : 1
COMMENT     : 2
TOOL        : 1
FILE-HEADER : 1
ORIGIN      : 1
CHANNEL     : 21
NO-FORMAT   : 1
PARAMETER   : 167

Unknown objects
--
280-FRAMESTEP-INFO : 1


In [8]:
dlis_df = []    # Conterá os dlis_df_dict respectivos aos poços

# Curvas de perfis que não serão utilizadas
curvas_nao_utilizadas = ['WF11', 'AHV', 'AHVT', 'BHV', 'BHVT', 'CS', 'FCPS', 'HDCN', 'HMCN', 'ITT', 'ITTT', 'LSPD', 'MMK', 'NCPS', 'SP']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for poco in dli_dict.values():
    try:
        # Armazenando as curvas que serão utilizadas em uma lista
        curvas_utilizadas = [
            channel.name                                    # Os elementos da lista serão os nomes das curvas
            for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
            if channel.name not in curvas_nao_utilizadas    # As curvas que não utilizaremos não serão armazenadas na lista
        ]


        curvas = poco.frames[0].curves()

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df.append(pd.DataFrame(curvas[curvas_utilizadas]))
    except:
        pass

In [9]:
# Criando um dicionário para associar nomes aos elementos de dlis_df
dlis_df_dict = {
    'p-47' : dlis_df[0],
    'p-48' : dlis_df[1],
    'p-51' : dlis_df[2],
    'p-53' : dlis_df[3],
    'p-55' : dlis_df[4],
    'p-57' : dlis_df[5]
}

In [10]:
dlis_df_dict['p-47']

,TDEP,GR,TENS,BS,NPHI,CALI,RHOB,DRHO,PE,HDRS,HMRS,DTC
0,374.899994,-999.250000,345.178406,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
1,374.799994,-999.250000,346.090210,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
2,374.699994,-999.250000,348.874512,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3,374.599994,-999.250000,348.379608,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
4,374.499994,-999.250000,349.917908,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
...,...,...,...,...,...,...,...,...,...,...,...,...
3595,15.399989,55.795601,910.639893,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3596,15.299989,52.518200,910.807007,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3597,15.199989,48.504501,909.179626,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3598,15.099989,45.021198,909.942383,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25


In [11]:
from aux_datapreprocess import *

## Remoção de Colunas

In [12]:
colunas_para_remover = ['CALI', 'RHOB', 'DRHO', 'PE', 'HDRS', 'HMRS', 'DTC']
dlis_df_dict['p-47'] = remove_colunas(dlis_df_dict['1847'], colunas_para_remover)

In [13]:
dlis_df_dict['p-47']

,TDEP,GR,TENS,BS,NPHI
0,374.899994,-999.250000,345.178406,8.5,-999.25
1,374.799994,-999.250000,346.090210,8.5,-999.25
2,374.699994,-999.250000,348.874512,8.5,-999.25
3,374.599994,-999.250000,348.379608,8.5,-999.25
4,374.499994,-999.250000,349.917908,8.5,-999.25
...,...,...,...,...,...
3595,15.399989,55.795601,910.639893,8.5,-999.25
3596,15.299989,52.518200,910.807007,8.5,-999.25
3597,15.199989,48.504501,909.179626,8.5,-999.25
3598,15.099989,45.021198,909.942383,8.5,-999.25


## Renomeação de Colunas

In [14]:
dlis_df_dict['p-47'] = renomeiaColuna(dlis_df_dict['1847'], "GR", "gamma-ray")

In [15]:
dlis_df_dict['p-47']

,TDEP,gamma-ray,TENS,BS,NPHI
0,374.899994,-999.250000,345.178406,8.5,-999.25
1,374.799994,-999.250000,346.090210,8.5,-999.25
2,374.699994,-999.250000,348.874512,8.5,-999.25
3,374.599994,-999.250000,348.379608,8.5,-999.25
4,374.499994,-999.250000,349.917908,8.5,-999.25
...,...,...,...,...,...
3595,15.399989,55.795601,910.639893,8.5,-999.25
3596,15.299989,52.518200,910.807007,8.5,-999.25
3597,15.199989,48.504501,909.179626,8.5,-999.25
3598,15.099989,45.021198,909.942383,8.5,-999.25


## Transformação NPHI

In [16]:
dlis_df_dict['p-53']

,TDEP,DTC,HDRS,HMRS,GR,CALI,NPHI,RHOB,DRHO,PE,TENS,BSZ
0,319.000000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1065.134399,8.50
1,318.900000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1340.839844,8.50
2,318.800000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1580.882080,8.50
3,318.700000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1798.774292,8.50
4,318.600000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1948.277588,8.50
...,...,...,...,...,...,...,...,...,...,...,...,...
3102,8.799995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25
3103,8.699995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25
3104,8.599995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25
3105,8.499995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25


In [17]:
nphi_transform(dlis_df_dict, ['p-47', 'p-53'])

In [18]:
dlis_df_dict['p-53']

,TDEP,DTC,HDRS,HMRS,GR,CALI,NPHI,RHOB,DRHO,PE,TENS,BSZ
0,319.000000,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,1065.134399,8.50
1,318.900000,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,1340.839844,8.50
2,318.800000,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,1580.882080,8.50
3,318.700000,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,1798.774292,8.50
4,318.600000,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,1948.277588,8.50
...,...,...,...,...,...,...,...,...,...,...,...,...
3102,8.799995,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,-999.250000,-999.25
3103,8.699995,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,-999.250000,-999.25
3104,8.599995,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,-999.250000,-999.25
3105,8.499995,-999.25,-999.25,-999.25,-999.25,-999.25,-99925.0,-999.25,-999.25,-999.25,-999.250000,-999.25


## Remoção de Pontos com Falha

In [12]:
dlis_df_dict['p-53']

,TDEP,DTC,HDRS,HMRS,GR,CALI,NPHI,RHOB,DRHO,PE,TENS,BSZ
0,319.000000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1065.134399,8.50
1,318.900000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1340.839844,8.50
2,318.800000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1580.882080,8.50
3,318.700000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1798.774292,8.50
4,318.600000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,1948.277588,8.50
...,...,...,...,...,...,...,...,...,...,...,...,...
3102,8.799995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25
3103,8.699995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25
3104,8.599995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25
3105,8.499995,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.250000,-999.25


In [16]:
pontos_com_falha = {
    'p-47': [250, 360],
    'p-48': [75, 210],
    'p-51': [60, 260],
    'p-53': [90, 221],
    'p-55': [64, 215],
    'p-57': [90, 320]
}

In [21]:
#dlis_df_dict['1847'].drop(dlis_df_dict['1847'][dlis_df_dict['1847']['TDEP'] > 300].index)

In [17]:
# Chamando a função
remove_pontos_com_falha(dlis_df_dict, pontos_com_falha)

In [18]:
dlis_df_dict['p-53']

,TDEP,DTC,HDRS,HMRS,GR,CALI,NPHI,RHOB,DRHO,PE,TENS,BSZ
980,220.999999,85.049835,10.950796,9.112442,44.597023,8.580480,0.165457,2.428133,-0.015682,4.602115,1609.141235,8.5
981,220.899999,86.915825,11.790182,9.760889,43.457493,8.589926,0.164601,2.425839,-0.016865,4.618337,1619.851318,8.5
982,220.799999,88.787437,11.485640,9.609058,43.871868,8.586134,0.163804,2.422934,-0.018172,4.664258,1634.729492,8.5
983,220.699999,92.809738,8.418729,7.365464,45.943741,8.559573,0.184878,2.427014,-0.019844,4.715289,1633.567871,8.5
984,220.599999,95.129929,6.886818,6.126797,48.119205,8.559573,0.203857,2.415779,-0.022114,4.697705,1618.269653,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...
2285,90.499997,198.503937,7.147364,6.389296,95.461456,8.711792,0.191479,2.411309,0.034687,4.497918,1471.055664,8.5
2286,90.399997,198.503937,12.763572,12.349573,72.722672,8.741086,0.147211,2.525461,0.028277,4.764666,1442.796509,8.5
2287,90.299997,151.259842,13.338861,13.308826,63.969017,8.732605,0.142141,2.550199,0.018576,4.775152,1429.798218,8.5
2288,90.199997,151.259842,11.085132,11.028458,62.259724,8.731671,0.150344,2.555811,0.013780,4.671879,1433.620117,8.5
